## First, run this cell to set up paths and import dependencies

In [ ]:
import os

from tqdm import tqdm

if not os.path.exists(r"./notebooks"):
    %cd ..

from src.audio_processor import AudioProcessor
from src.audio_dataset_processor import DAPSDatasetProcessor
from src.data_processing import SOAAudioClips, save_mean_std, compute_mean_std_from_images
from src.dataset_analysis import duration_statistics
from src.config import VALID_ACCESS_LABELS, TRAIN_DIR, TEST_DIR, VAL_DIR, DATA_DIR, DATASET_DIR

os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

## 1. Splitting Allowed `.wav` Files

We worked with the [DAPS dataset](https://zenodo.org/records/4660670), which contains multiple directories of `.wav` files. The dataset consists of audio recordings of 5 scripts read by 20 different speakers. The directories in the dataset vary based on two main factors: **room** and **recording device** (used for augmentation purposes).

1. **Allowed Directories**:  
   We first specify which directories are allowed to be included in the dataset. Each directory is identified by its **room** and **recording device** information. The directories contain the `.wav` files, which are grouped by scripts and speakers.

2. **Dataset Splitting**:  
   Using the `DAPSDatasetProcessor` class, we systematically discover and split the `.wav` files across the following datasets:
   - **Training Dataset**: The set of files used for training the model. (60%)
   - **Validation Dataset**: The set of files used for validating model performance during training. (20%)
   - **Test Dataset**: The set of files used to evaluate the model's final performance. (20%)

   The splitting process ensures that the same script will not appear in both the training and test datasets, which is important for preventing data leakage. In implementation it takes fom every 5 recordings of combination of **room**, **recording device** and **speaker** first 3 recordings are added to training dataset split, the next one to validate split and the last recording to test dataset split. It ensures that there cannot be a situation, when the same recording of different augmentation is present in the different splits of the input dataset.

3. **Balancing Classes**:  
   If there is an imbalance between the classes (i.e., one class has more samples than the other), we use `batch_sampler` in the `DataLoader` for training. The `batch_sampler` is configured to **undersample** the major class to balance the datasets. This technique ensures that the model isn't biased toward the more frequent class.

4. **Organizing Data for Use**:  
   Once the datasets are split and balanced, the files are organized and stored in a separate directories for every dataset split.

This step ensures that we have properly structured and balanced datasets for training, validation, and testing, which will help to train a robust and generalizable model.


In [ ]:
allowed_directories=['ipadflat_confroom1', 'ipadflat_office1', 'ipad_balcony1', 'ipad_bedroom1', 'ipad_confroom1', 'ipad_confroom2', 'ipad_livingroom1', 'ipad_office1', 'ipad_office2', 'iphone_balcony1', 'iphone_bedroom1', 'iphone_livingroom1']
dataset_processor = DAPSDatasetProcessor(DATA_DIR, VALID_ACCESS_LABELS, allowed_directories)
dataset_processor.compute_statistics()
train_set, validate_set, test_set = dataset_processor.get_datasets()

## 2. Duration statistics of .wav files
We calculate input clips duration statistics for each dataset split.

In [ ]:
soa_train_full_clips = SOAAudioClips(train_set)
soa_test_full_clips = SOAAudioClips(validate_set)
soa_val_full_clips = SOAAudioClips(test_set)

print("\nDataset Statistics:")
print("Training set:")
print(duration_statistics(soa_train_full_clips.clips))

print("Validation set:")
print(duration_statistics(soa_test_full_clips.clips))

print("Test set:")
print(duration_statistics(soa_val_full_clips.clips))

## 3. Splitting Audio into Few-Second Subclips

In this section, we process full audio clips by splitting them into smaller subclips of a fixed duration. This step ensures that the input to our CNN has a consistent size. Additionally, we filter subclips to retain only those with significant speech content, as determined by `webrtcvad`.


1. **Random Subclip Generation**:  
   Each full clip is randomly segmented into smaller subclips of `subclip_length` seconds. This ensures variation and better generalization in the dataset.

2. **Subclip Count Calculation**:  
   For each audio clip, the number of subclips is determined using the formula:
   $$
   \text{num\_segments} = \left \lfloor{\frac{\text{audio\_length}}{\text{subclip\_length}} \cdot \text{factor}}\right \rfloor
   $$
   - `audio_length`: Total duration of the input audio clip (in seconds).
   - `subclip_length`: Duration of each subclip (in seconds).
   - `factor`: A multiplier (e.g., 1.1) to slightly oversample the number of subclips.

3. **Speech Filtering**:  
   Only subclips where more than 50% of the audio contains speech (as detected by `webrtcvad`) are saved. This ensures that the CNN training data primarily contains meaningful speech content.

Then after processing we save log mel grayscale spectrograms into separate directories for every dataset.

In [ ]:
audio_processor = AudioProcessor()

print("Preprocessed Train Dataset:")
duration_stats = audio_processor.process_audio_clips(soa_train_full_clips, TRAIN_DIR)
print(duration_stats)

print("\nPreprocessed Validation Dataset:")
duration_stats = audio_processor.process_audio_clips(soa_val_full_clips, VAL_DIR)
print(duration_stats)

print("\nPreprocessed Test Dataset:")
duration_stats = audio_processor.process_audio_clips(soa_test_full_clips, TEST_DIR)
print(duration_stats)

## 4. Mean and Standard Deviation of the Training Dataset

Normalization is a crucial preprocessing step when training neural networks. For our dataset of grayscale images, normalization ensures that the pixel intensity values are transformed to have a consistent distribution. This step improves the stability of the training process, smooths the loss function surface, and facilitates optimization by making it easier for the model to converge to the global minimum.

### Mathematical Description

Given a training dataset of grayscale images $ \mathbf{X} = \{ \mathbf{I}_1, \mathbf{I}_2, \ldots, \mathbf{I}_N \} $, where each $ \mathbf{I}_i $ is a 2D matrix representing an image with pixel intensity values scaled to the range \([0.0, 1.0]\), normalization is performed as follows:

1. **Compute the mean and standard deviation** of the pixel values across all training images:
   $$
   \mu = \frac{1}{N \cdot H \cdot W} \sum_{i=1}^N \sum_{h=1}^H \sum_{w=1}^W \mathbf{I}_i(h, w),
   $$
   $$
   \sigma = \sqrt{\frac{1}{N \cdot H \cdot W} \sum_{i=1}^N \sum_{h=1}^H \sum_{w=1}^W \left(\mathbf{I}_i(h, w) - \mu\right)^2},
   $$
   where:
   - $ \mu $ is the mean pixel intensity across all training images,
   - $ \sigma $ is the standard deviation of pixel intensities,
   - $ H $ and $ W $ are the height and width of each image,
   - $ \mathbf{I}_i(h, w) $ represents the pixel intensity at position $(h, w)$ in image $ \mathbf{I}_i $.

2. **Normalize each image** $ \mathbf{I}_i $ using the computed mean and standard deviation:
   $$
   \mathbf{I}_i^\text{normalized}(h, w) = \frac{\mathbf{I}_i(h, w) - \mu}{\sigma}.
   $$

Here, $ \mathbf{I}_i^\text{normalized} $ is the normalized image where pixel values have zero mean and unit variance.

### Implementation Details
- **Dataset Type**: Grayscale images with pixel intensity values in the range $[0.0, 1.0]$.
- **Normalization Scope**: The mean $ \mu $ and standard deviation $ \sigma $ are computed **only from the training dataset**. 
- **Storage**: The computed $ \mu $ and $ \sigma $ values are saved to a JSON file located alongside the dataset's image directories. This ensures consistent normalization of the validation and test datasets during evaluation.

### Importance of Normalization
Normalization improves the training process by ensuring a consistent scale across the input data, which helps prevent numerical instability and accelerates model convergence. For an example of the impact of normalization on the performance of a convolutional neural network (CNN), see [this visualization](https://wandb.ai/mytkom-warsaw-university-of-technology/iml/reports/Effect-of-normalization-input-for-TutorialCNN--VmlldzoxMDUxMTI1OQ?accessToken=s67utpfjryb4um1240bd56i51zo5oy2bj0gbaqqz79z3hnnabkub1rdhsamhwd2v).

In [ ]:
mean, std = compute_mean_std_from_images(TRAIN_DIR)
print(f"Mean: {mean}, Standard deviation: {std}")
save_mean_std(mean, std, f"{DATASET_DIR}/scaling_params.json")